<a href="https://colab.research.google.com/github/Santiago-R/aupa.ai/blob/main/lm-hackers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [A hacker's guide to Language Models](https://colab.research.google.com/github/fastai/lm-hackers/blob/main/lm-hackers.ipynb#scrollTo=0b017bfc-5be0-4e41-9fa1-9f685c3b0de5)


## Initial restart

In [1]:
import os
import importlib
a_spec = importlib.util.find_spec('accelerate')
b_spec = importlib.util.find_spec('bitsandbytes')
found = a_spec is not None and b_spec is not None

if found: print('Dependencies installed in previous run ✔️')
else:
    !pip install accelerate -qq
    !pip install bitsandbytes -qq

    # New
    import locale
    locale.getpreferredencoding = lambda: "UTF-8"
    !pip install auto-gptq -qq
    !pip install optimum -qq

    print('\nDependency installation requires restart --> killing runtime 💀')

Dependencies installed in previous run ✔️


In [2]:
if not found:
    # Kill runtime if required
    os.kill(os.getpid(), 9)

## Setup

In [3]:
from google.colab import drive
drive.mount('/content/drive')  # , force_remount=True)

Mounted at /content/drive


In [4]:
# import tokenize
# from io import BytesIO

In [5]:
# from pathlib import Path
# path = Path("/content/drive/MyDrive/LLM")

## The OpenAI API

In [6]:
# Load OpenAI api key as environvent variable (from Drive's api_keys.env)
!pip install python-dotenv -qq
from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/LLM/api_keys.env')

True

In [7]:
!pip install openai -qq
from openai import ChatCompletion, Completion

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 872.0 kB/s eta 0:00:00


#### Sys prompt

In [8]:
aussie_sys = "You are an Aussie LLM that uses Aussie slang and analogies whenever possible."
question = "What is money?"

c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": aussie_sys},
              {"role": "user", "content": question}])

- [Model options](https://platform.openai.com/docs/models)

In [9]:
def response(c):
    try:
        return c['choices'][0]['message']['content']
    except KeyError:
        return c['choices'][0]['text']

In [10]:
response(c)

"Money, mate, is like the fuel that keeps the financial engine revving. It's an essential tool we use to exchange goods and services. Think of it as the grease that keeps the economic wheels turning. Whether it's cold hard cash or digital payments, money makes the world go round, just like how a kangaroo bounces around the outback. So, next time you're fumbling through your wallet, remember that money is the language of the economy, and without it, things would be as topsy-turvy as a didgeridoo played by a wombat."

#### Usage

In [11]:
print(c.usage)

{
  "prompt_tokens": 31,
  "completion_tokens": 119,
  "total_tokens": 150
}


In [12]:
0.002 / 1000 * 150  # GPT 3.5

0.0003

In [13]:
0.03 / 1000 * 150  # GPT 4

0.0045

#### askgpt

In [14]:
c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "system", "content": aussie_sys},
              {"role": "user", "content": "What is money?"},
              {"role": "assistant", "content": "Well, mate, money is like kangaroos actually."},
              {"role": "user", "content": "Really? In what way?"}])

In [15]:
response(c)

"Yeah, let me break it down for you. Just like kangaroos hop around and carry little joeys in their pouches, money hops around the economy and carries value. Money is a medium of exchange that we use to buy goods and services. It's like the fuel that keeps the economic machine running, mate. Without money, things would get pretty messy, just like a mob of kangaroos bouncing chaotically in all directions. So, in short, money is the lifeblood of the economy, just like kangaroos are the lifeblood of the Aussie outback."

In [16]:
def askgpt(user, system=None, model="gpt-3.5-turbo", **kwargs):
    msgs = []
    if system: msgs.append({"role": "system", "content": system})
    msgs.append({"role": "user", "content": user})
    return ChatCompletion.create(model=model, messages=msgs, **kwargs)

In [17]:
response(askgpt('What is the meaning of life?', system=aussie_sys))

"Ah, mate, the meaning of life is like searching for the Holy Grail or trying to catch a wave that never ends. It's a deep philosophical question, but I reckon it's all about finding your own purpose and making the most out of this crazy journey we call existence. For some, it's about love, family, or making a difference in the world. For others, it's all about having a bloody good time and enjoying the ride. The key is to figure out what truly makes you happy and go after it, like snagging the perfect wave or finding that elusive snag at a BBQ. That's what gives life its true meaning, my friend."

- [Limits](https://platform.openai.com/docs/guides/rate-limits/what-are-the-rate-limits-for-our-api)

Created by Bing:

In [18]:
def call_api(prompt, model="gpt-3.5-turbo"):
    msgs = [{"role": "user", "content": prompt}]
    try: return ChatCompletion.create(model=model, messages=msgs)
    except openai.error.RateLimitError as e:
        retry_after = int(e.headers.get("retry-after", 60))
        print(f"Rate limit exceeded, waiting for {retry_after} seconds...")
        time.sleep(retry_after)
        return call_api(params, model=model)

In [19]:
response(call_api("What's the world's funniest joke? Has there ever been any scientific analysis?"))

'Determining the world\'s funniest joke is subjective, as humor varies greatly from person to person and across different cultures. However, there have been attempts to analyze humor scientifically. The most famous of these efforts is the LaughLab experiment conducted in 2002 by psychologist Richard Wiseman. The experiment involved collecting jokes and rating them based on how funny they were.\n\nOver 1.5 million jokes were submitted and rated by people from around the world. One of the jokes that emerged as a favorite was the following:\n\n"Two hunters are out in the woods when one of them collapses. He doesn\'t seem to be breathing, and his eyes are glazed. The other guy whips out his phone and calls emergency services. He gasps, \'My friend is dead! What can I do?\' The operator says, \'Calm down, I can help. First, let\'s make sure he\'s dead.\' There is a silence; then, a gunshot is heard. Back on the phone, the guy says, \'Okay, now what?\'"\n\nWhile this joke was popular among t

In [20]:
c = Completion.create(prompt="Australian Jeremy Howard is ",
                      model="gpt-3.5-turbo-instruct", echo=True)  # logprobs=5)

In [21]:
response(c)

'Australian Jeremy Howard is  now starting a Silicon Valley-based startup with the aim of making artificial intelligence more accessible'

## Create our own code interpreter

In [22]:
from pydantic import create_model
import inspect, json
from inspect import Parameter

#### Example with sums function

In [23]:
def sums(a:int, b:int=1):
    "Adds a + b"
    return a + b

In [24]:
def schema(f):
    kw = {n:(o.annotation, ... if o.default==Parameter.empty else o.default)
          for n,o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).schema()
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

In [25]:
schema(sums)

{'name': 'sums',
 'description': 'Adds a + b',
 'parameters': {'title': 'Input for `sums`',
  'type': 'object',
  'properties': {'a': {'title': 'A', 'type': 'integer'},
   'b': {'title': 'B', 'default': 1, 'type': 'integer'}},
  'required': ['a']}}

In [26]:
c = askgpt("Use the `sum` function to solve this: What is 6+3?",
           system = "You must use the `sum` function instead of adding yourself.",
           functions=[schema(sums)])

In [27]:
c

<OpenAIObject chat.completion id=chatcmpl-86nWa0uDsbjt7DWtwZuwIVjNDhIXF at 0x7c95988731a0> JSON: {
  "id": "chatcmpl-86nWa0uDsbjt7DWtwZuwIVjNDhIXF",
  "object": "chat.completion",
  "created": 1696631032,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "sums",
          "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 22,
    "total_tokens": 105
  }
}

In [28]:
m = c.choices[0].message
m

<OpenAIObject at 0x7c9566f2aac0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "sums",
    "arguments": "{\n  \"a\": 6,\n  \"b\": 3\n}"
  }
}

In [29]:
k = m.function_call.arguments
print(k)

{
  "a": 6,
  "b": 3
}


In [30]:
funcs_ok = {'sums', 'python'}

In [31]:
def call_func(c):
    fc = c.choices[0].message.function_call
    if fc.name not in funcs_ok: return print(f'Not allowed: {fc.name}')
    f = globals()[fc.name]
    return f(**json.loads(fc.arguments))

In [32]:
call_func(c)

9

In [33]:
import ast

def run(code):
    tree = ast.parse(code)
    last_node = tree.body[-1] if tree.body else None

    # If the last node is an expression, modify the AST to capture the result
    if isinstance(last_node, ast.Expr):
        tgts = [ast.Name(id='_result', ctx=ast.Store())]
        assign = ast.Assign(targets=tgts, value=last_node.value)
        tree.body[-1] = ast.fix_missing_locations(assign)

    ns = {}
    exec(compile(tree, filename='<ast>', mode='exec'), ns)
    return ns.get('_result', None)

In [34]:
run("""
a=1
b=2
a+b
""")

3

In [35]:
def python(code:str, safe_mode:bool=False):
    "Return result of executing `code` using python. If execution not permitted, returns `#FAIL#`"
    if safe_mode:
        go = input(f'Proceed with execution?\n```\n{code}\n```\n')
        if go.lower()!='y': return '#FAIL#'
    return run(code)

In [36]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[schema(python)])

In [37]:
def code_response(c, repl=True):
    txt_out = response(c)
    if txt_out == None: txt_out = ''
    if 'function_call' not in c['choices'][0]['message'].keys():
        print(txt_out)
        return  # No code output
    code = c['choices'][0]['message']['function_call']['arguments']
    if code[0] == '{': code = json.loads(code)['code']
    txt_out += f'\n==========\n{code}\n==========\n>>> '
    result = run(code)
    print(txt_out)
    return result

In [38]:
c

<OpenAIObject chat.completion id=chatcmpl-86nWbTG3gFOdAZN8gmHMJGb19nyJB at 0x7c9566f0f0b0> JSON: {
  "id": "chatcmpl-86nWbTG3gFOdAZN8gmHMJGb19nyJB",
  "object": "chat.completion",
  "created": 1696631033,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "python",
          "arguments": "{\n  \"code\": \"import math\\nmath.factorial(12)\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 21,
    "total_tokens": 103
  }
}

In [39]:
call_func(c)

479001600

In [40]:
code_response(c)


import math
math.factorial(12)
>>> 


479001600

#### Using Python's exec & eval

In [41]:
exec_schema = {
    'name': 'exec',
    'description': 'Execute the given source Python code',
    'parameters': {
        'title': 'Input for `exec`',
        'type': 'object',
        'properties': {'source': {'title': 'S', 'type': 'string'}},
        'required': ['source']}}

In [45]:
def code_response(c, repl=True):
    txt_out = response(c)
    if txt_out == None: txt_out = ''
    if 'function_call' not in c['choices'][0]['message'].keys():
        print(txt_out)
        return  # No code output
    code = c['choices'][0]['message']['function_call']['arguments']
    if code[0] == '{': code = json.loads(code)['source']
    txt_out += f'\n==========\n{code}\n==========\n>>> '
    if repl:
        code_body = '\n'.join(code.split('\n')[:-1])
        code_footer = code.split('\n')[-1]
        exec(code_body, locals())
        result = eval(code_footer, locals())
        txt_out += str(result)
    else:
        exec(code, locals())
        result = None
    print(txt_out)
    return result

In [46]:
c = askgpt("What is 12 factorial?",
           system = "Use python for any required computations.",
           functions=[exec_schema])

In [47]:
factorial_result = code_response(c, repl=True)


import math
math.factorial(12)
>>> 479001600


In [48]:
factorial_result

479001600

###### Using result in later calls

In [49]:
c = ChatCompletion.create(
    model="gpt-3.5-turbo",
    functions=[exec_schema],
    messages=[{"role": "user", "content": "What is 12 factorial?"},
              {"role": "function", "name": "exec", "content": str(factorial_result)}])

In [50]:
print(response(c))

12 factorial, denoted as 12!, is equal to 479,001,600.


###### And we didn't break its basic use!

In [51]:
c = askgpt("What is the capital of France?",
           system = "Use python for any required computations.",
           functions=[exec_schema])

In [52]:
code_response(c)

The capital of France is Paris.


## PyTorch and Huggingface

In [54]:
!pip install transformers -qq
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

- [HF leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard)
- [fasteval](https://fasteval.github.io/FastEval/)

In [55]:
mn = "meta-llama/Llama-2-7b-hf"

In [56]:
# !huggingface-cli login
# # Model access needs granted at https://huggingface.co/meta-llama/Llama-2-7b-hf

# TODO: Login in environment variable

In [57]:
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [58]:
tokr = AutoTokenizer.from_pretrained(mn)
prompt = "Jeremy Howard is a "
toks = tokr(prompt, return_tensors="pt")

In [59]:
toks

{'input_ids': tensor([[    1,  5677,  6764, 17430,   338,   263, 29871]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [60]:
tokr.batch_decode(toks['input_ids'])

['<s> Jeremy Howard is a ']

In [61]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

CPU times: user 10.6 s, sys: 958 ms, total: 11.5 s
Wall time: 23.7 s


tensor([[    1,  5677,  6764, 17430,   338,   263, 29871, 29941, 29953,  1629,
          2030,  9870,   319,  2801,   515, 16198, 29892,  1570,  4275, 13706,
         29892,  8314]])

In [62]:
tokr.batch_decode(res)

['<s> Jeremy Howard is a 36 year old Australian Actor from Sydney, New South Wales, Australia']

In [63]:
# # OutOfMemoryError
# model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

In [64]:
# %%time
# res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
# res

In [65]:
model = AutoModelForCausalLM.from_pretrained('TheBloke/Llama-2-7b-Chat-GPTQ', device_map=0, torch_dtype=torch.float16)

In [66]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


CPU times: user 729 ms, sys: 17.4 ms, total: 746 ms
Wall time: 749 ms


tensor([[    1,  5677,  6764, 17430,   338,   263, 29871, 29941, 29945, 29899,
          6360, 29899,  1025,   767,   515,   278,  3303,  3900,  1058,   471,
         24383,   297]])

In [67]:
mn = 'TheBloke/Llama-2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

OutOfMemoryError: ignored

In [ ]:
%%time
res = model.generate(**toks.to("cuda"), max_new_tokens=15).to('cpu')
res

In [ ]:
def gen(p, maxlen=15, sample=True):
    toks = tokr(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample).to('cpu')
    return tokr.batch_decode(res)

In [ ]:
gen(prompt, 50)

[StableBeluga-7B](https://huggingface.co/stabilityai/StableBeluga-7B)

In [ ]:
mn = "stabilityai/StableBeluga-7B"
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.bfloat16)

In [ ]:
sb_sys = "### System:\nYou are Stable Beluga, an AI that follows instructions extremely well. Help as much as you can.\n\n"

In [ ]:
def mk_prompt(user, syst=sb_sys): return f"{syst}### User: {user}\n\n### Assistant:\n"

In [ ]:
ques = "Who is Jeremy Howard?"

In [ ]:
gen(mk_prompt(ques), 150)

[OpenOrca/Platypus 2](https://huggingface.co/Open-Orca/OpenOrca-Platypus2-13B)

In [ ]:
mn = 'TheBloke/OpenOrca-Platypus2-13B-GPTQ'
model = AutoModelForCausalLM.from_pretrained(mn, device_map=0, torch_dtype=torch.float16)

In [ ]:
def mk_oo_prompt(user): return f"### Instruction: {user}\n\n### Response:\n"

In [ ]:
gen(mk_oo_prompt(ques), 150)

In [ ]:
!pip install wikipedia-api
from wikipediaapi import Wikipedia

In [ ]:
wiki = Wikipedia('JeremyHowardBot/0.0', 'en')
jh_page = wiki.page('Jeremy_Howard_(entrepreneur)').text
jh_page = jh_page.split('\nReferences\n')[0]

In [ ]:
print(jh_page[:500])

In [ ]:
len(jh_page.split())

In [ ]:
# def mk_prompt_context(question, context):
#     return f"""Answer the question with the help of the provided context.\n\n## Context\n\n{context}\n\n## Question\n\n{question}"""

In [ ]:
res = gen(mk_prompt(ques_ctx), 300)

In [ ]:
print(res[0].split('### Assistant:\n')[1])

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
emb_model = SentenceTransformer("BAAI/bge-small-en-v1.5", device=0)

In [ ]:
jh = jh_page.split('\n\n')[0]
print(jh)

In [ ]:
tb_page = wiki.page('Tony_Blair').text.split('\nReferences\n')[0]

In [ ]:
tb = tb_page.split('\n\n')[0]
print(tb[:380])

In [ ]:
q_emb,jh_emb,tb_emb = emb_model.encode([ques,jh,tb], convert_to_tensor=True)

In [ ]:
tb_emb.shape

In [ ]:
import torch.nn.functional as F

In [ ]:
F.cosine_similarity(q_emb, jh_emb, dim=0)

In [ ]:
F.cosine_similarity(q_emb, tb_emb, dim=0)

### Private GPTs

- [Sooo many](https://github.com/h2oai/h2ogpt/blob/main/docs/README_LangChain.md#what-is-h2ogpts-langchain-integration-like)

## Fine tuning

In [ ]:
import datasets

[knowrohit07/know_sql](https://huggingface.co/datasets/knowrohit07/know_sql)

In [ ]:
ds = datasets.load_dataset('knowrohit07/know_sql', revision='f33425d13f9e8aab1b46fa945326e9356d6d5726')

In [ ]:
ds

In [ ]:
trn = ds['train']
trn[3]

`accelerate launch -m axolotl.cli.train sql.yml`

In [ ]:
tst = dict(**trn[3])
tst['question'] = 'Get the count of competition hosts by theme.'
tst

In [ ]:
fmt = """SYSTEM: Use the following contextual information to concisely answer the question.

USER: {}
===
{}
ASSISTANT:"""

In [ ]:
def sql_prompt(d): return fmt.format(d["context"], d["question"])

In [ ]:
print(sql_prompt(tst))

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
ax_model = '/home/jhoward/git/ext/axolotl/qlora-out'

In [ ]:
tokr = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')

In [ ]:
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-2-7b-hf',
                                             torch_dtype=torch.bfloat16, device_map=0)
model = PeftModel.from_pretrained(model, ax_model)
model = model.merge_and_unload()
model.save_pretrained('sql-model')

In [ ]:
toks = tokr(sql_prompt(tst), return_tensors="pt")

In [ ]:
res = model.generate(**toks.to("cuda"), max_new_tokens=250).to('cpu')

In [ ]:
print(tokr.batch_decode(res)[0])

## [llama.cpp](https://github.com/abetlen/llama-cpp-python)

[TheBloke/Llama-2-7b-Chat-GGUF](https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF)

In [ ]:
!pip install llama_cpp_python -qq
from llama_cpp import Llama

In [ ]:
llm = Llama(model_path="content/llamacpp/llama-2-7b-chat.Q4_K_M.gguf")

In [ ]:
output = llm("Q: Name the planets in the solar system? A: ", max_tokens=32, stop=["Q:", "\n"], echo=True)

In [ ]:
print(output['choices'])